In [ ]:
# Extractive QA  : 책에서 문제의 답을 찾는다
# 각 단어가 답의 시작일 확률 계산  start_logits
# 각 단어가 답의 끝일 확률 계산    end_logits
# 가장 확률이 높은 구간 선택       argmax

# Extractive QA 시각화
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 모델 로드
MODEL_NAME = 'distilbert-base-cased-distilled-squad'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

In [ ]:
question = "What is the capital of France?"
context = "Paris is the capital and largest city of France. The city has a population of 2.1 million."

In [ ]:
# 토크나이제이션
inputs = tokenizer(question,context,return_tensors='pt')

In [ ]:
tokenizer.decode(inputs['input_ids'][0])

In [ ]:
# 토큰 리스트
tokens = tokenizer.convert_ids_to_tokens( inputs['input_ids'][0] )
print(tokens[:10])

In [ ]:
# 예측
with torch.no_grad():
  outputs = model(**inputs)
outputs

In [ ]:
# logits를 확률로 변환
start_probs = torch.softmax(outputs.start_logits, dim = -1)[0].numpy()
end_probs = torch.softmax(outputs.end_logits, dim = -1)[0].numpy()
# 답변 추출
answer_start = np.argmax(start_probs)
answer_end = np.argmax(end_probs) + 1
answer_tokens = tokens[answer_start:answer_end]
answer = tokenizer.convert_tokens_to_string(answer_tokens)

print(f'모델의 답변 : {answer}')
print(f'시작위치 : {answer_start}  토큰 : {tokens[answer_start]}')
print(f'종료위치 : {answer_end}  토큰 : {tokens[answer_end-1]}')
print(f'신뢰도 : {start_probs[answer_start] * end_probs[answer_end-1]:.4f}')

In [ ]:
# 상위 5개 후보출력
# 시작위치 상위 5개
top_start = np.argsort(-start_probs)[:5]
for i,index in enumerate(top_start,1):
  print(f'시작위치 : {i}  토큰 : {tokens[index]} 확률 : {start_probs[index]}')
# 종료위치 상위 5개
print()
top_end = np.argsort(-end_probs)[:5]
for i,index in enumerate(top_end,1):
  print(f'종료위치 : {i}  토큰 : {tokens[index]} 확률 : {end_probs[index]}')

In [ ]:
len(start_probs), len(tokens)

In [ ]:
fig, (ax1,ax2)  = plt.subplots(2,1,figsize=(15,10))
ax1.bar(range(len(tokens)),start_probs)
ax2.bar(range(len(tokens)),end_probs)
plt.show()


In [ ]:
# 결론 : 확률이 고르게 분산 -> 답을 못찾은 경우, 특정구간에 집중되면 확신있는 답을 찾음